In [12]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.metrics import confusion_matrix, f1_score


In [13]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32


In [14]:
train_dir = r"C:\Users\USER\Desktop\capstone\archive\data\train"
val_dir = r"C:\Users\USER\Desktop\capstone\archive\data\val"

In [15]:
train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

val_dataset = image_dataset_from_directory(val_dir,
                                           shuffle=True,
                                           batch_size=BATCH_SIZE,
                                           image_size=IMG_SIZE)


Found 33984 files belonging to 4 classes.
Found 6400 files belonging to 4 classes.


In [16]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))


In [17]:
base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = True

In [18]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4, activation='softmax')  # 4 classes for Alzheimer's
])


In [19]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [20]:
epoch_data = []

# Train and save metrics after each epoch
for epoch in range(10):
    print(f"Epoch {epoch+1}/10")
    history = model.fit(train_dataset, validation_data=val_dataset, epochs=1)

    # Evaluate model on validation data
    y_true = []
    y_pred = []
    
    for images, labels in val_dataset:
        preds = model.predict(images)
        y_pred.extend(np.argmax(preds, axis=1))
        y_true.extend(labels.numpy())

    # Calculate confusion matrix and metrics
    cm = confusion_matrix(y_true, y_pred)
    TP = np.diag(cm)
    FP = np.sum(cm, axis=0) - TP
    FN = np.sum(cm, axis=1) - TP
    TN = cm.sum() - (TP + FP + FN)
    f1 = f1_score(y_true, y_pred, average='weighted')

    epoch_data.append({
        'Epoch': epoch + 1,
        'Loss': history.history['loss'][0],
        'Accuracy': history.history['accuracy'][0],
        'Val_Loss': history.history['val_loss'][0],
        'Val_Accuracy': history.history['val_accuracy'][0],
        'F1_Score': f1,
        'True_Positive': sum(TP),
        'True_Negative': sum(TN),
        'False_Positive': sum(FP),
        'False_Negative': sum(FN)
    })

    # Save metrics to Excel
    df = pd.DataFrame(epoch_data)
    df.to_excel("training_metrics_inceptionv3.xlsx", index=False)
    print(f"Metrics for Epoch {epoch + 1} saved to training_metrics_inceptionv3.xlsx")

Epoch 1/10
1062/1062 ━━━━━━━━━━━━━━━━━━━━ 5170s 5s/step - accuracy: 0.6680 - loss: 0.7514 - val_accuracy: 0.7033 - val_loss: 1.2255
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 944ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 966ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 882ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 935ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 977ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 855ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 926ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 978ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 914ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 917ms/step
1/1 ━━━━━━

In [21]:
model.save("final_alzheimer_inceptionV3_model.keras")

In [22]:
import numpy as np
import joblib
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [24]:
teacher_model = tf.keras.models.load_model("final_alzheimer_inceptionV3_model.keras")

In [26]:
train_dir = r"C:\Users\USER\Desktop\capstone\archive\data\train"
val_dir = r"C:\Users\USER\Desktop\capstone\archive\data\val"
img_size = (224, 224)
batch_size = 32

In [27]:
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode='sparse', shuffle=False
)
val_generator = datagen.flow_from_directory(
    val_dir, target_size=img_size, batch_size=batch_size, class_mode='sparse', shuffle=False
)

Found 33984 images belonging to 4 classes.
Found 6400 images belonging to 4 classes.


In [28]:
features = []

soft_labels = []

true_labels = []

In [29]:
def extract_features(generator):
    temp_features = []
    temp_soft_labels = []
    temp_true_labels = []
    for batch_images, batch_labels in generator:
        batch_preds = teacher_model.predict(batch_images)
        temp_features.extend(batch_preds)
        temp_soft_labels.extend(np.argmax(batch_preds, axis=1))
        temp_true_labels.extend(batch_labels)
        if len(temp_features) >= len(generator.filenames):
            break
    return np.array(temp_features), np.array(temp_soft_labels), np.array(temp_true_labels)

In [30]:
train_features, train_soft_labels, train_true_labels = extract_features(train_generator)
val_features, val_soft_labels, val_true_labels = extract_features(val_generator)

1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3

In [31]:
svm_model = SVC(probability=True)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
knn_model = KNeighborsClassifier(n_neighbors=5)

In [32]:
from sklearn.ensemble import VotingClassifier
ensemble_model = VotingClassifier(
    estimators=[('svm', svm_model), ('rf', rf_model), ('knn', knn_model)],
    voting='soft'
)

In [33]:
ensemble_model.fit(train_features, train_true_labels)

VotingClassifier(estimators=[('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('knn', KNeighborsClassifier())],
                 voting='soft')

In [34]:
val_predictions = ensemble_model.predict(val_features)

In [35]:
from sklearn.metrics import classification_report
print("Validation Accuracy:", accuracy_score(val_true_labels, val_predictions))
print("Classification Report:\n", classification_report(val_true_labels, val_predictions))

Validation Accuracy: 0.98328125
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.99      1.00       896
         1.0       1.00      1.00      1.00        64
         2.0       0.97      1.00      0.98      3200
         3.0       1.00      0.96      0.98      2240

    accuracy                           0.98      6400
   macro avg       0.99      0.99      0.99      6400
weighted avg       0.98      0.98      0.98      6400



In [36]:
svm_model.fit(train_features, train_soft_labels)
rf_model.fit(train_features, train_soft_labels)
knn_model.fit(train_features, train_soft_labels)

svm_predictions = svm_model.predict(val_features)
rf_predictions = rf_model.predict(val_features)
knn_predictions = knn_model.predict(val_features)

svm_accuracy = accuracy_score(val_true_labels, svm_predictions)
rf_accuracy = accuracy_score(val_true_labels, rf_predictions)
knn_accuracy = accuracy_score(val_true_labels, knn_predictions)

print(f"SVM Accuracy: {svm_accuracy * 100:.2f}%")
print(f"Random Forest Accuracy: {rf_accuracy * 100:.2f}%")
print(f"KNN Accuracy: {knn_accuracy * 100:.2f}%")

SVM Accuracy: 95.19%
Random Forest Accuracy: 95.17%
KNN Accuracy: 95.17%


In [37]:
svm_val_predictions = svm_model.predict(val_features)
rf_val_predictions = rf_model.predict(val_features)
knn_val_predictions = knn_model.predict(val_features)

svm_val_accuracy = accuracy_score(val_true_labels, svm_val_predictions)
rf_val_accuracy = accuracy_score(val_true_labels, rf_val_predictions)
knn_val_accuracy = accuracy_score(val_true_labels, knn_val_predictions)

print(f"SVM Validation Accuracy: {svm_val_accuracy * 100:.2f}%")
print(f"Random Forest Validation Accuracy: {rf_val_accuracy * 100:.2f}%")
print(f"KNN Validation Accuracy: {knn_val_accuracy * 100:.2f}%")

SVM Validation Accuracy: 95.19%
Random Forest Validation Accuracy: 95.17%
KNN Validation Accuracy: 95.17%


In [ ]:
svm_val_predictions=svm_model.predict(val_features)
rf_val_predictions=rf_model.predict(val_features)
knn_val_predictions=knn_model.predict(val_features)


In [ ]:
rf_val_predictions=rf.model.predict(val_features)
knn_val_predictions=knn_model.predict(val_features)
svm_val_predictions=svmz_model.predict(val_features)